In [1]:
import cuejax as cue 
import numpy as np

In [2]:
emul = cue.Emulator()

In [3]:
# default model parameters:
# (this is also the order that the theta vector is assumed to be in 
# when you do emul.predict_lines(theta=theta) or emul.predict_cont(theta=theta))

default_params = {
        'ionspec_index1': 19.7, 
        'ionspec_index2': 5.3, 
        'ionspec_index3': 1.6, 
        'ionspec_index4': 0.6,
        'ionspec_logLratio1': 3.9, 
        'ionspec_logLratio2': 0.01, 
        'ionspec_logLratio3': 0.2,
        'gas_logu': -2.5, 
        'gas_logq':49.1,
        'gas_logn': 2,
        'gas_logz': 0., 
        'gas_logno': 0., 
        'gas_logco': 0.,
        'gas_logqion': 49.1, 
        }

Becuase `cuejax` precision is hard-coded to be `jnp.float32`, there are small ($\sim 1$% difference) errors that are introduced when using `cuejax` vs. `cue`.

In [4]:
lines = emul.predict_lines(**default_params)
for i,line in enumerate(lines[0]): print(emul.model.line_names[i],'=',line,'Lsun')

H 1 923.150A = 7.659444 Lsun
H 1 926.226A = 8.040534 Lsun
H 1 930.748A = 8.594249 Lsun
H 1 937.804A = 9.372741 Lsun
H 1 949.743A = 10.611199 Lsun
H 1 972.537A = 12.416246 Lsun
H 1 1025.72A = 15.929629 Lsun
H 1 1215.67A = 37812.285 Lsun
He 2 1640.41A = 5.231199 Lsun
O 3 1660.81A = 8.408174 Lsun
O 3 1666.15A = 24.67568 Lsun
Ne 3 1814.56A = 1.9457256 Lsun
Al 3 1854.72A = 10.935661 Lsun
Al 3 1862.79A = 6.7575336 Lsun
C 3 1906.68A = 170.0914 Lsun
C 3 1908.73A = 114.015755 Lsun
N 2 2142.77A = 16.086689 Lsun
O 3 2320.95A = 10.039957 Lsun
C 2 2323.50A = 10.31927 Lsun
C 2 2324.69A = 18.113535 Lsun
C 2 2325.40A = 97.04918 Lsun
C 2 2326.93A = 62.796505 Lsun
C 2 2328.12A = 20.470238 Lsun
O 2 2471.00A = 60.38719 Lsun
Al 2 2660.35A = 69.93217 Lsun
Al 2 2669.15A = 47.492004 Lsun
Mg 2 2795.53A = 276.67557 Lsun
Mg 2 2802.71A = 137.20174 Lsun
Ar 3 3109.18A = 5.654469 Lsun
Ne 3 3342.18A = 0.6425164 Lsun
S 3 3721.63A = 22.158825 Lsun
O 2 3726.03A = 1797.0118 Lsun
O 2 3728.81A = 2448.7188 Lsun
H 1 3797.90A

In [5]:
wave = np.logspace(3,4,1000) # output wavelength grid for continuum in units of restframe AA (1,000 - 10,000 AA)
# you can also specify unit = 'Lsun/Hz' or unit = 'erg/s/Hz'
cont = emul.predict_cont(wave,unit='Lsun/Hz',**default_params) 